In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import csv
import math
from pymoo.problems import get_problem
from pymoo.util.plotting import plot
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter
from scipy.stats import ranksums

In [2]:
def dtlz1(x,n, var):
    array = np.array(x)
    pf = get_problem("dtlz1", n_var = var, n_obj=n)
    return pf.evaluate(array)

def dtlz2(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz2", n_var = var, n_obj=n)
    return pf.evaluate(array)

def dtlz5(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz5", n_var = var, n_obj=n)
    return pf.evaluate(array)

def dtlz7(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz7", n_var = var, n_obj=n)
    return pf.evaluate(array)

def idtlz1(x,n, var):
    array = np.array(x)
    pf = get_problem("dtlz1", n_var = var, n_obj=n)
    return -pf.evaluate(array)

def idtlz2(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz2", n_var = var, n_obj=n)
    return -pf.evaluate(array)

def idtlz5(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz5", n_var = var, n_obj=n)
    return -pf.evaluate(array)

def idtlz7(x,n,var):
    array = np.array(x)
    pf = get_problem("dtlz7", n_var = var, n_obj=n)
    return -pf.evaluate(array)

print(dtlz1([.5,.5], 2, 2))
print(idtlz1([0,0], 2, 2))

[0.25 0.25]
[ -0. -13.]


In [3]:
def NSGA(func, funcn, epochs, var, pop, lowEnd, highEnd):
    ################## Create first generation with random numbers between
    ################## lowEnd and highEnd
    population_matrix = []
    children_matrix = []
    non_dominated = []
    children_results = []
    results = []
    NDResults = []
    best = []
    i = 0
    while i < pop:
        individual = []
        ########################## Create individual of var size
        for j in range(0,var):
            individual.append(random.uniform(lowEnd[j], highEnd[j]))
        population_matrix.append(individual)
        ########################## Evaluate individual in all functions
        results.append(func(individual, funcn, var).tolist())
        i += 1
    ########## Sort on non-domination
    NDLayers = layers(population_matrix.copy(), results.copy())
    ############## Create children matrix with tournament selection, SBX and Mutation
    Layer0Res = NDLayers[0][1].copy()
    tot = []
    sole = False
    for j in range(len(Layer0Res)):
        tot.append(0)
        for k in range(len(Layer0Res[0])):
            tot[j] += Layer0Res[j][k]
    if(len(tot) > 1):
        indx1 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
        Layer0Res.pop(tot.index(min(tot)))
        tot.pop(tot.index(min(tot)))
        indx2 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
    else:
        indx1 = 0
        sole = True
    ############## Perform crossover
    best.append(NDLayers[0][1])
    #best.append((NDLayers[0][0][indx1],NDLayers[0][1][indx1]))
    j = 0
    while j < int(pop/2):
        l = NDLayers[0][0][indx1]
        if (sole):
            q = NDLayers[1][0][0]
        else:
            q = NDLayers[0][0][indx2]
        cross = SBX([l,q],lowEnd,highEnd)
        children_matrix.append(cross[0])
        children_matrix.append(cross[1])
        children_results.append(func(cross[0], funcn, var).tolist())
        children_results.append(func(cross[1], funcn, var).tolist())
        j += 1

    ############### Perform Mutation
    for j in range(0,pop):
        r = random.uniform(0,1)
        if(r <= .1):
            n = random.randint(0,var-1)
            children_matrix[j][n] = PM(children_matrix[j][n], 100, lowEnd[n], highEnd[n])
            children_results[j] = func(children_matrix[j], funcn, var).tolist()

    whole_pop_matrix = population_matrix + children_matrix
    whole_res_matrix = results + children_results

    for i in range(0,epochs):
        ################ Obtain all of the non-dominated results by layers and graph them per generation
        NDLayers = layers(whole_pop_matrix.copy(), whole_res_matrix.copy())
        ################ Eliminate objects from worse categories until only pop elements remain
        left = pop
        myLayers = []
        for j in range(len(NDLayers)):
            myLayers.append([[],[]])
            for k in range(len(NDLayers[j][0])):
                if(left > 0):
                    myLayers[j][0].append(NDLayers[j][0][k])
                    myLayers[j][1].append(NDLayers[j][1][k])
                    left -= 1
                else:
                    break
            if(left == 0):
                break

        NDLayers = myLayers.copy()



        ################# Assign new population
        population_matrix = []
        results = []
        for j in range(0, len(NDLayers)):
            for k in range(0, len(NDLayers[j][0])):
                population_matrix.append(NDLayers[j][0][k])
                results.append(NDLayers[j][1][k])


        """
        colors = ["red", "violet", "cyan", "green"]
        dimension = len(NDLayers[0][1][0])
        if (dimension > 2):
            fig = plt.figure()
            ax = fig.add_subplot(projection='3d')
        else:
            fig = plt.figure()
            ax = fig.add_subplot()
        j = 0
        while j < len(NDLayers):
            paretoPlot(NDLayers[j][1], dimension, colors[j%len(colors)], ax)
            j += 1
        """
        ################ Find the two biggest crowding distance of layer 1
        sole = False
        Layer0Res = NDLayers[0][1].copy()
        Layer0Res = sorted(Layer0Res, key = lambda x: x[0])
        distance = []
        for j in range(len(Layer0Res)):
            distance.append(0)
        for j in range(1,len(Layer0Res)-1):
            dist_matrix = []
            for k in range(0, funcn):
                dist_matrix.append((Layer0Res[j+1][k] - Layer0Res[j-1][k])**2)
            distance[j] = (sum(dist_matrix))**(1/2)
        if(len(distance) > 1):
            indx1 = NDLayers[0][1].index(Layer0Res[distance.index(max(distance))])
            Layer0Res[distance.index(max(distance))] = -1
            distance[distance.index(max(distance))] = -1
            indx2 = NDLayers[0][1].index(Layer0Res[distance.index(max(distance))])
        else:
            indx1 = 0
            sole = True
        """
        Layer0Res = NDLayers[0][1].copy()
        tot = []
        sole = False
        for j in range(len(Layer0Res)):
            tot.append(0)
            for k in range(len(Layer0Res[0])):
                tot[j] += Layer0Res[j][k]
        if(len(tot) > 1):
            indx1 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
            Layer0Res.pop(tot.index(min(tot)))
            tot.pop(tot.index(min(tot)))
            indx2 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
        else:
            indx1 = 0
            sole = True

        """
        ############## Perform crossover
        best.append(NDLayers[0][1])
        #best.append((NDLayers[0][0][indx1],NDLayers[0][1][indx1]))
        j = 0
        while j < int(pop/2):
            l = NDLayers[0][0][indx1]

            if (sole):
                q = NDLayers[1][0][0]
            else:
                q = NDLayers[0][0][indx2]
            cross = SBX([l,q],lowEnd,highEnd)
            children_matrix[j*2] = cross[0]
            children_matrix[j*2 + 1] = cross[1]
            children_results[j*2] = func(cross[0].copy(), funcn, var).tolist()
            children_results[j*2 + 1] = func(cross[1].copy(), funcn, var).tolist()
            j += 1

        ############### Perform Mutation
        for j in range(0,pop):
            r = random.uniform(0,1)
            if(r <= .1):
                n = random.randint(0,var-1)
                children_matrix[j][n] = PM(children_matrix[j][n], 100+i, lowEnd[n], highEnd[n])
                children_results[j] = func(children_matrix[j], funcn, var).tolist()


        whole_pop_matrix = population_matrix + children_matrix
        whole_res_matrix = results + children_results



    return best

In [4]:
def SBX(parents, low, high):
    nc = 50
    u = random.uniform(0,1)
    b = 0
    bi = 0
    if (u<=.5):
        b = (2*u)**(1/(nc + 1))
    else:
        b = (1/(2*(1-u)))**(1/(nc + 1))

    if (b<=1):
        bi = .5*(nc + 1)*b**nc
    else:
        bi = .5*(nc + 1)*(1/b**(nc + 1))



    l = (np.add(parents[0],parents[1])/2) - b*abs(np.subtract(parents[1],parents[0])/2)
    #l = ((1 + bi) * np.array(parents[0]))/2 + (1-bi) * np.array(parents[1])
    q = (np.add(parents[0],parents[1])/2) + b*abs(np.subtract(parents[1],parents[0])/2)
    #q = ((1 - bi) * np.array(parents[0]))/2 + (1-bi) * np.array(parents[1])

    for i in range(len(l)):
        if (l[i] < low[i]):
            l[i] = low[i]
        elif (l[i] > high[i]):
            l[i] = high[i]
        if (q[i] < low[i]):
            q[i] = low[i]
        elif (q[i] > high[i]):
            q[i] = high[i]

    return l.tolist(),q.tolist()

def PM(value, nm, low, high):
    u = random.uniform(0,1)
    d = min(value - low, high - value)/(high - low)
    dq = 0
    if (u<=.5):
        dq = ((2*u + (1-2*u)*(1-d)**(nm+1))**(1/(nm + 1))) - 1
    else:
        dq = 1 - ((2*(1-u) + 2*(u-.5)*(1-d)**(nm+1))**(1/(nm + 1)))


    value = value + dq*(high-low)
    if (value < low):
        value = low
    elif (value > high):
        value = high

    return value

In [5]:
def NSDE(func, funcn, epochs, var, pop, lowEnd, highEnd):
    ################## Create first generation with random numbers between
    ################## lowEnd and highEnd
    population_matrix = []
    children_matrix = []
    non_dominated = []
    children_results = []
    results = []
    NDResults = []
    best = []
    i = 0
    while i < pop:
        individual = []
        ########################## Create individual of var size
        for j in range(0,var):
            individual.append(random.uniform(lowEnd[j], highEnd[j]))
        population_matrix.append(individual)
        ########################## Evaluate individual in all functions
        results.append(func(individual, funcn, var).tolist())
        i += 1
    ########## Sort on non-domination
    NDLayers = layers(population_matrix.copy(), results.copy())
    ############## Create children matrix with tournament selection, and differential crossover/mutation
    Layer0Res = NDLayers[0][1].copy()
    tot = []
    sole = False
    for j in range(len(Layer0Res)):
        tot.append(0)
        for k in range(len(Layer0Res[0])):
            tot[j] += Layer0Res[j][k]
    if(len(tot) > 2):
        indx1 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
        Layer0Res[tot.index(min(tot))] = 1000000000000
        tot[tot.index(min(tot))] = 10000000000
        indx2 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
        Layer0Res[tot.index(min(tot))] = 1000000000000
        tot[tot.index(min(tot))] = 10000000000
        indx3 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
    else:
        indx1 = 0
        sole = True
    ############## Perform crossover and mutation
    best.append(NDLayers[0][1])
    #best.append((NDLayers[0][0][indx1],NDLayers[0][1][indx1]))
    j = 0
    while j < pop:
        r = random.randint(0,pop-1)
        l = NDLayers[0][0][indx1]
        if (sole):
            q = NDLayers[1][0][0]
            m = NDLayers[1][0][r % len(NDLayers[1][0])]
        else:
            q = NDLayers[0][0][indx2]
            m = NDLayers[0][0][(r - indx3) % len(NDLayers[0][0])]
        res = differentialMutation(l,q,m, lowEnd, highEnd)
        prob = differentialCrossover(pop)
        individual1 = []
        individual2 = []
        half = int(var/2)
        for k in range(half):
            individual1.append(l[k])
            individual2.append(res[k])
        for k in range(half, var):
            individual1.append(res[k])
            individual2.append(l[k])
        if(prob < 0 or pop < prob):
            children_matrix.append(individual1)
        else:
            children_matrix.append(individual2)
        children_results.append(func(population_matrix[j], funcn, var).tolist())
        j += 1



    whole_pop_matrix = population_matrix + children_matrix
    whole_res_matrix = results + children_results

    for i in range(0,epochs):
        ################ Obtain all of the non-dominated results by layers and graph them per generation
        NDLayers = layers(whole_pop_matrix.copy(), whole_res_matrix.copy())
        ################ Eliminate objects from worse categories until only pop elements remain
        left = pop
        myLayers = []
        for j in range(len(NDLayers)):
            myLayers.append([[],[]])
            for k in range(len(NDLayers[j][0])):
                if(left > 0):
                    myLayers[j][0].append(NDLayers[j][0][k])
                    myLayers[j][1].append(NDLayers[j][1][k])
                    left -= 1
                else:
                    break
            if(left == 0):
                break


        NDLayers = myLayers.copy()

        ################# Assign new population
        population_matrix = []
        results = []
        for j in range(0, len(NDLayers)):
            for k in range(0, len(NDLayers[j][0])):
                population_matrix.append(NDLayers[j][0][k])
                results.append(NDLayers[j][1][k])


        """
        colors = ["red", "violet", "cyan", "green"]
        dimension = len(NDLayers[0][1][0])
        if (dimension > 2):
            fig = plt.figure()
            ax = fig.add_subplot(projection='3d')
        else:
            fig = plt.figure()
            ax = fig.add_subplot()
        j = 0
        while j < len(NDLayers):
            paretoPlot(NDLayers[j][1], dimension, colors[j%len(colors)], ax)
            j += 1
        """
        ################ Find the two biggest crowding distance of layer 1
        sole = False
        Layer0Res = NDLayers[0][1].copy()
        Layer0Res = sorted(Layer0Res, key = lambda x: x[0])
        distance = []
        for j in range(len(Layer0Res)):
            distance.append(0)
        for j in range(1,len(Layer0Res)-1):
            dist_matrix = []
            for k in range(0, funcn):
                dist_matrix.append((Layer0Res[j+1][k] - Layer0Res[j-1][k])**2)
            distance[j] = (sum(dist_matrix))**(1/2)
        if(len(distance) > 2):
            indx1 = NDLayers[0][1].index(Layer0Res[distance.index(max(distance))])
            Layer0Res[distance.index(max(distance))] = -1
            distance[distance.index(max(distance))] = -1
            indx2 = NDLayers[0][1].index(Layer0Res[distance.index(max(distance))])
            Layer0Res[distance.index(max(distance))] = -1
            distance[distance.index(max(distance))] = -1
            indx3 = NDLayers[0][1].index(Layer0Res[distance.index(max(distance))])
        else:
            indx1 = 0
            sole = True
        """
        Layer0Res = NDLayers[0][1].copy()
        tot = []
        sole = False
        for j in range(len(Layer0Res)):
            tot.append(0)
            for k in range(len(Layer0Res[0])):
                tot[j] += Layer0Res[j][k]
        if(len(tot) > 2):
            indx1 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
            Layer0Res[tot.index(min(tot))] = 1000000000000
            tot[tot.index(min(tot))] = 10000000000
            indx2 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
            Layer0Res[tot.index(min(tot))] = 1000000000000
            tot[tot.index(min(tot))] = 10000000000
            indx3 = NDLayers[0][1].index(Layer0Res[tot.index(min(tot))])
        else:
            indx1 = 0
            sole = True
        """

        best.append(NDLayers[0][1])
        #best.append((NDLayers[0][0][indx1],NDLayers[0][1][indx1]))
        ############## Perform crossover and mutation
        j = 0
        while j < pop:
            r = random.randint(0,pop-1)
            l = NDLayers[0][0][indx1]
            if (sole):
                q = NDLayers[1][0][0]
                m = NDLayers[1][0][r % len(NDLayers[1][0])]
            else:
                q = NDLayers[0][0][indx2]
                m = NDLayers[0][0][(r - indx3) % len(NDLayers[0][0])]
            res = differentialMutation(l,q,m, lowEnd, highEnd)
            prob = differentialCrossover(pop)
            individual1 = []
            individual2 = []
            half = int(var/2)
            for k in range(half):
                individual1.append(l[k])
                individual2.append(res[k])
            for k in range(half, var):
                individual1.append(res[k])
                individual2.append(l[k])
            if(prob < 0 or pop < prob):
                children_matrix[j] = individual1
            else:
                children_matrix[j] = individual2
            children_results[j] = func(children_matrix[j], funcn, var).tolist()
            j += 1



        whole_pop_matrix = population_matrix + children_matrix
        whole_res_matrix = results + children_results

    return best

In [6]:
def differentialCrossover(size):
    J = 0
    j = size - 1
    for i in range(j):
        J = J and (j+1)
        j = (j+1) %size
        if(J == size):
            break
    return J


def differentialMutation(parent1, parent2, parent3, low, high):
    value = parent1 + .5*(np.subtract(parent2,parent3))
    for i in range(len(parent1)):
        if (value[i] < low[i]):
            value[i] = low[i]
        elif (value[i] > high[i]):
            value[i] = high[i]

    return value.tolist()

In [7]:
def paretoFront(pop, res):
    i = 0
    dom = False
    while i < len(res):
        current = res[i]
        j = 0
        while j < len(res):
            compare = res[j]
            for k in range(len(res[j])):
                if(current[k] > compare[k]):
                    dom = False
                    break
                elif(current[k] == compare[k]):
                    continue
                dom = True
            if(dom):
                del pop[j]
                del res[j]
                dom = False
                if(j < i):
                    i -=1
                continue
            j += 1
        i += 1
    return pop, res

In [8]:
def paretoPlot(points, dimensions, color, subplot, k):
    x = []
    for i in range(dimensions):
        x.append(list(map(lambda x : x[i], points)))
    if (dimensions > 2):
        if(k == "b"):
            subplot.scatter(x[0],x[1],[y - .1 for y in x[2]],zorder = 0,c = color, alpha = .5)
        else:
            subplot.scatter(x[0],x[1],x[2],zorder = 10,c = color)
    else:
        if(k == "b"):
            subplot.plot(x[0],x[1],zorder = 0,c = color, alpha = .5)
        else:
            subplot.scatter(x[0],x[1],zorder = 10,c = color)


In [9]:
def layers(pop, res):
    tot_pop = pop.copy()
    tot_res = res.copy()
    #colors = ["red", "violet", "blue", "green", "silver", "gold", "orange"]
    layers = []

    while True:
        if (len(tot_res) == 0):
            break
        layer_pop, layer_res = paretoFront(tot_pop.copy(),tot_res.copy())
        layers.append([layer_pop,layer_res])
        for j in range(len(layer_res)):
            tot_res.remove(layer_res[j])
            tot_pop.remove(layer_pop[j])
    return layers




In [10]:
def organize(m):
    finalList = []
    for i in range(len(m)):
      for j in range(len(m[i])):
        finalList.append(m[i][j])
    z = [None] * len(finalList)
    NDLayers = layers(z, finalList)

    Layer0Res = NDLayers[0][1].copy()
    Layer0Res = sorted(Layer0Res, key = lambda x:x[0])
    tot = []
    for j in range(len(Layer0Res)):
        tot.append(0)
        for k in range(len(Layer0Res[0])):
            tot[j] += Layer0Res[j][k]

    return Layer0Res, Layer0Res[tot.index(min(tot))]

def graph(m1,m2,m3,title):
    z1 = [None] * len(m1)
    z2 = [None] * len(m2)
    NDLayers1 = layers(z1, m1)
    NDLayers2 = layers(z2, m2)
    colors = ["red", "violet", "cyan", "green"]
    dimension = len(NDLayers1[0][1][0])
    if (dimension > 2):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.view_init(elev=10, azim=60, roll=0)
    else:
        fig = plt.figure()
        ax = fig.add_subplot()
    print(title, " has ", len(NDLayers1[0][1]), " on the pareto line in NSGA")
    print(title, " has ", len(NDLayers2[0][1]), " on the pareto line in NSDE")
    paretoPlot(NDLayers1[0][1], dimension, colors[0], ax, "n")
    paretoPlot(NDLayers2[0][1], dimension, colors[1], ax, "n")
    paretoPlot(m3, dimension, colors[2], ax, "b")
    ax.legend(["NSGA","NSDE","Optimal"])
    ax.set_title(title)

def graphr(m1,m2,title):
    z1 = [None] * len(m1)
    z2 = [None] * len(m2)
    NDLayers1 = layers(z1, m1)
    NDLayers2 = layers(z2, m2)
    colors = ["red", "violet", "cyan", "green"]
    dimension = len(NDLayers1[0][1][0])
    if (dimension > 2):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.view_init(elev=10, azim=60, roll=0)
    else:
        fig = plt.figure()
        ax = fig.add_subplot()
    print(title, " has ", len(NDLayers1[0][1]), " on the pareto line in NSGA")
    print(title, " has ", len(NDLayers2[0][1]), " on the pareto line in NSDE")
    paretoPlot(NDLayers1[0][1], dimension, colors[0], ax, "n")
    paretoPlot(NDLayers2[0][1], dimension, colors[1], ax, "n")
    ax.legend(["NSGA","NSDE","Optimal"])
    ax.set_title(title)


In [11]:
def weighted_sum(val, weights):
    totSum = 0
    for i in range(len(val)):
        totSum += val[i] * weights[i]
    return totSum

def r2(val, optimal):
    r2 = 0       
    for i in range(len(val)-1):
        r2 += weighted_sum(optimal[i], [1/len(optimal[i])]*len(optimal[i])) - weighted_sum(val[i], [1/len(val[i])]*len(val[i]))
    return r2

def RSE(point1, point2):
    s = 1

    return (1/(euclidean_distance(point1, point2)+.000001)**(s))

def euclidean_distance(point1, point2):
    squared_distances = [(a - b) ** 2 for a, b in zip(point1, point2)]
    sum_of_squares = sum(squared_distances)
    distance = math.sqrt(sum_of_squares)
    return distance


In [ ]:
real, dif, realR2, difR2, realRSE, difRSE = [[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30]
funcs = [dtlz1,dtlz2,dtlz5,dtlz7]
text = ["dtlz1","dtlz2","dtlz5","dtlz7"]
for i in range(0,30):
    for j in range(0,4):
        n = j+4
        funcn = int((n)/2 - ((-1)**n - 1)/-4)
        realHist = NSGA(funcs[j],funcn, 20, funcn, 20, [0]*funcn, [1]*funcn)
        difHist = NSDE(funcs[j],funcn, 20, funcn, 20, [0]*funcn, [1]*funcn)
        tempr, x = organize(realHist.copy())
        tempd, y = organize(difHist.copy())
        real[j][i] = tempr
        dif[j][i] = tempd

        ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
        ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
        d = [get_problem("dtlz1").pareto_front(ref_dirs1),get_problem("dtlz2").pareto_front(ref_dirs2),get_problem("dtlz5", n_obj = 3, n_var = 3).pareto_front(), get_problem("dtlz7",n_obj = 3, n_var = 3).pareto_front()]

        realR2[j][i] = r2(tempr.copy(), d[j])

        d[0] = ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempd)-1)
        d[1] = ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempd)-1)

        difR2[j][i] = r2(tempd.copy(), d[j])

        realRSE[j][i] = RSE(tempr[0],tempr[-1])
        difRSE[j][i] = RSE(tempd[0],tempd[-1])
    print("Final de generacion", i)

#return final pareto front, return best fitness individual
temp1, bestr1 = organize(real[0])
temp2, bestr2 = organize(real[1])
temp3, bestr3 = organize(real[2])
temp4, bestr4 = organize(real[3])
temp5, bestd1 = organize(dif[0])
temp6, bestd2 = organize(dif[1])
temp7, bestd3 = organize(dif[2])
temp8, bestd4 = organize(dif[3])

print("Best result of dtlz1:\n", "NSGAII:\n", "\tFunction Result: ", bestr1, " R2 mean: ", np.mean(realR2[0]), " R2 std: ", np.std(realR2[0]), " RSE mean:", np.mean(realRSE[0]), "RSE std: ", np.std(realRSE[0]), "\n", "NSDE:\n", "\tFunction Result: ", bestd1, " R2 mean: ", np.mean(difR2[0]), " R2 std: ", np.std(difR2[0]), " RSE mean:", np.mean(difRSE[0]), "RSE std: ", np.std(difRSE[0]), "\n")
print("Best result of dtlz2:\n", "NSGAII:\n", "\tFunction Result: ", bestr2, " R2 mean: ", np.mean(realR2[1]), " R2 std: ", np.std(realR2[1]), " RSE mean:", np.mean(realRSE[1]), "RSE std: ", np.std(realRSE[1]), "\n", "NSDE:\n", "\tFunction Result: ", bestd2, " R2 mean: ", np.mean(difR2[1]), " R2 std: ", np.std(difR2[1]), " RSE mean:", np.mean(difRSE[1]), "RSE std: ", np.std(difRSE[1]), "\n")
print("Best result of dtlz5:\n", "NSGAII:\n", "\tFunction Result: ", bestr3, " R2 mean: ", np.mean(realR2[2]), " R2 std: ", np.std(realR2[2]), " RSE mean:", np.mean(realRSE[2]), "RSE std: ", np.std(realRSE[2]), "\n", "NSDE:\n", "\tFunction Result: ", bestd3, " R2 mean: ", np.mean(difR2[2]), " R2 std: ", np.std(difR2[2]), " RSE mean:", np.mean(difRSE[2]), "RSE std: ", np.std(difRSE[2]), "\n")
print("Best result of dtlz7:\n", "NSGAII:\n", "\tFunction Result: ", bestr4, " R2 mean: ", np.mean(realR2[3]), " R2 std: ", np.std(realR2[3]), " RSE mean:", np.mean(realRSE[3]), "RSE std: ", np.std(realRSE[3]), "\n", "NSDE:\n", "\tFunction Result: ", bestd4, " R2 mean: ", np.mean(difR2[3]), " R2 std: ", np.std(difR2[3]), " RSE mean:", np.mean(difRSE[3]), "RSE std: ", np.std(difRSE[3]), "\n")
print(ranksums(realR2[0],difR2[0])[0], ranksums(realRSE[0], difRSE[0])[0])
print(ranksums(realR2[1],difR2[1])[0], ranksums(realRSE[1], difRSE[1])[0])
print(ranksums(realR2[2],difR2[2])[0], ranksums(realRSE[2], difRSE[2])[0])
print(ranksums(realR2[3],difR2[3])[0], ranksums(realRSE[3], difRSE[3])[0])

Final de generacion 0
Final de generacion 1
Final de generacion 2
Final de generacion 3
Final de generacion 4
Final de generacion 5
Final de generacion 6
Final de generacion 7
Final de generacion 8


In [ ]:
ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
d = [get_problem("dtlz1").pareto_front(ref_dirs1),get_problem("dtlz2").pareto_front(ref_dirs2),get_problem("dtlz5", n_obj = 3, n_var = 3).pareto_front(), get_problem("dtlz7",n_obj = 3, n_var = 3).pareto_front()]

graph(temp1, temp5, d[0], "dtlz1")
graph(temp2, temp6, d[1], "dtlz2")
graph(temp3, temp7, d[2], "dtlz5")
graph(temp4, temp8, d[3], "dtlz7")




In [ ]:
real, dif, realR2, difR2, realRSE, difRSE = [[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30],[[None]*30,[None]*30,[None]*30,[None]*30]
funcs = [idtlz1,idtlz2,idtlz5,idtlz7]
text = ["idtlz1","idtlz2","idtlz5","itlz7"]
for i in range(0,30):
    for j in range(0,4):
        n = j+4
        funcn = int((n)/2 - ((-1)**n - 1)/-4)
        realHist = NSGA(funcs[j],funcn, 20, funcn, 20, [0]*funcn, [1]*funcn)
        difHist = NSDE(funcs[j],funcn, 20, funcn, 20, [0]*funcn, [1]*funcn)

        tempr, x = organize(realHist.copy())
        tempd, y = organize(difHist.copy())
        real[j][i] = tempr
        dif[j][i] = tempd

        ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
        ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
        d = [get_problem("dtlz1").pareto_front(ref_dirs1),get_problem("dtlz2").pareto_front(ref_dirs2),get_problem("dtlz5", n_obj = 3, n_var = 2).pareto_front(), get_problem("dtlz7",n_obj = 3, n_var = 2).pareto_front()]

        realR2[j][i] = r2(tempr.copy(), -d[j])

        d[0] = ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempd)-1)
        d[1] = ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempd)-1)

        difR2[j][i] = r2(tempd.copy(), -d[j])

        realRSE[j][i] = RSE(tempr[0],tempr[-1])
        difRSE[j][i] = RSE(tempd[0],tempd[-1])
    print("Final de generacion", i)

temp1, bestr1 = organize(real[0])
temp2, bestr2 = organize(real[1])
temp3, bestr3 = organize(real[2])
temp4, bestr4 = organize(real[3])
temp5, bestd1 = organize(dif[0])
temp6, bestd2 = organize(dif[1])
temp7, bestd3 = organize(dif[2])
temp8, bestd4 = organize(dif[3])

print("Best result of idtlz1:\n", "NSGAII:\n", "\tFunction Result: ", bestr1, " R2 mean: ", np.mean(realR2[0]), " R2 std: ", np.std(realR2[0]), " RSE mean:", np.mean(realRSE[0]), "RSE std: ", np.std(realRSE[0]), "\n", "NSDE:\n", "\tFunction Result: ", bestd1, " R2 mean: ", np.mean(difR2[0]), " R2 std: ", np.std(difR2[0]), " RSE mean:", np.mean(difRSE[0]), "RSE std: ", np.std(difRSE[0]), "\n")
print("Best result of idtlz2:\n", "NSGAII:\n", "\tFunction Result: ", bestr2, " R2 mean: ", np.mean(realR2[1]), " R2 std: ", np.std(realR2[1]), " RSE mean:", np.mean(realRSE[1]), "RSE std: ", np.std(realRSE[1]), "\n", "NSDE:\n", "\tFunction Result: ", bestd2, " R2 mean: ", np.mean(difR2[1]), " R2 std: ", np.std(difR2[1]), " RSE mean:", np.mean(difRSE[1]), "RSE std: ", np.std(difRSE[1]), "\n")
print("Best result of idtlz5:\n", "NSGAII:\n", "\tFunction Result: ", bestr3, " R2 mean: ", np.mean(realR2[2]), " R2 std: ", np.std(realR2[2]), " RSE mean:", np.mean(realRSE[2]), "RSE std: ", np.std(realRSE[2]), "\n", "NSDE:\n", "\tFunction Result: ", bestd3, " R2 mean: ", np.mean(difR2[2]), " R2 std: ", np.std(difR2[2]), " RSE mean:", np.mean(difRSE[2]), "RSE std: ", np.std(difRSE[2]), "\n")
print("Best result of idtlz7:\n", "NSGAII:\n", "\tFunction Result: ", bestr4, " R2 mean: ", np.mean(realR2[3]), " R2 std: ", np.std(realR2[3]), " RSE mean:", np.mean(realRSE[3]), "RSE std: ", np.std(realRSE[3]), "\n", "NSDE:\n", "\tFunction Result: ", bestd4, " R2 mean: ", np.mean(difR2[3]), " R2 std: ", np.std(difR2[3]), " RSE mean:", np.mean(difRSE[3]), "RSE std: ", np.std(difRSE[3]), "\n")
print(ranksums(realR2[0],difR2[0])[0], ranksums(realRSE[0], difRSE[0])[0])
print(ranksums(realR2[1],difR2[1])[0], ranksums(realRSE[1], difRSE[1])[0])
print(ranksums(realR2[2],difR2[2])[0], ranksums(realRSE[2], difRSE[2])[0])
print(ranksums(realR2[3],difR2[3])[0], ranksums(realRSE[3], difRSE[3])[0])

In [ ]:
ref_dirs1 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
ref_dirs2 = get_reference_directions("das-dennis", 3, n_partitions=len(tempr)-1)
d = [get_problem("dtlz1").pareto_front(ref_dirs1),get_problem("dtlz2").pareto_front(ref_dirs2),get_problem("dtlz5", n_obj = 3, n_var = 2).pareto_front(), get_problem("dtlz7",n_obj = 3, n_var = 2).pareto_front()]

graphr(temp1, temp5, "idtlz1")
graphr(temp2, temp6, "idtlz2")
graphr(temp3, temp7, "idtlz5")
graphr(temp4, temp8, "idtlz7")